In [2]:
from typing import Protocol, Generic, Optional, Union, Any

In [3]:
inputs = [
    {
        "id": "first",
        "type": "int",
    },
    {
        "id": "second",
        "type": "float",
    }
]
outputs = [
    {
        "id": "output",
        "type": "str",
    }
]

def factory():
    exec(f"""def func(
         {", ".join([i["id"] + ": " + i["type"] for i in inputs])}
    ) -> {outputs[0]["type"]}:
        return str(first + second)
    """)
    return func

f = factory()
print(f(1, 2.0))

NameError: name 'func' is not defined

In [34]:
class SbgFile:
    pass

class CwlType:
    def __init__(self, id: str, type: str | list | dict, **kwds):
        self.id = id
        self.cwl_type = type
        for k, v in kwds.items():
            setattr(self, k, v)

    def __str__(self):
        type_ = CwlType.from_cwl_type(self.cwl_type)
        if hasattr(type_, "__origin__"):
            return f"{self.id}: {type_}"
        else:
            return f"{self.id}: {type_.__name__}"
    
    def check(self, val: Any) -> bool:
        return CwlType.check_type(val, self.cwl_type)
    
    def get_type(self) -> type:
        return CwlType.from_cwl_type(self.cwl_type)
    
    def optional(self, type_: None | type=None) -> bool:
        type_ = type_ or CwlType.from_cwl_type(self.cwl_type)
        try:
            types_ = type_.__args__
            if type(None) in types_:
                return True
            else:
                return any([self.optional(t) for t in types_])
        except AttributeError:
            # Optiona[...] === Union[type | None], meaning all optional parameters
            # will have an __args__ parameter. If missing, this parameter cannot be
            # optional.
            return False

    @staticmethod
    def from_cwl_type(cwl_type: str | list | dict) -> type:
        _TYPES = {
            "string": str,
            "boolean": bool,
            "int": int,
            "long": int,
            "float": float,
            "double": float,
            "null": type(None),
            "record": dict,
            "File": SbgFile
        }

        if isinstance(cwl_type, str):
            # named type
            optional = cwl_type.endswith("?")
            return Optional[_TYPES[cwl_type.strip("?")]] if optional else _TYPES[cwl_type]
        elif isinstance(cwl_type, list):
            # list of possible values
            return Union[tuple(CwlType.from_cwl_type(v) for v in cwl_type)]
        elif isinstance(cwl_type, dict):
            # complex types (array or enum)
            container_ = cwl_type["type"]
            optional = container_.endswith("?")
            type_ = {
                "array": lambda: list[CwlType.from_cwl_type(cwl_type["items"])],
                "enum": lambda: Any
            }[container_.strip("?")]()
            return Optional[type_] if optional else type_
        elif cwl_type is None:
            # null type
            return TYPES["null"]
        else:
            raise TypeError(f"Invalid type description: {cwl_type}")
    
    @staticmethod
    def check_type(val: Any, cwl_type: str | list | dict) -> bool:
        type_ = CwlType.from_cwl_type(cwl_type)
        try:
            return isinstance(val, type_)
        except TypeError:
            if hasattr(type_, "__origin__"):
                # list: check item type
                return (isinstance(val, type_.__origin__) and
                        all(CwlType.check_type(v, cwl_type["items"]) for v in val))
            else:
                # enum
                return val in cwl_type["symbols"]

In [35]:
str(CwlType(id="foo", type="record"))

'foo: dict'

In [47]:
CwlType(id="opt", type={"type": "enum?", "symbols": ["A", "B", "C"]}).optional()

True